### Created simple parameter : 0.4*OXY + 0.3*SECCHI + 0.3*CHLORA measuring water quality and tried predicting it on models provided at models.ipynb with some small modifications, using elements from fertilizers and silicate as X
### works ok mostly based on silicates,  but other matters too r2_score better at greater depths

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from zipfile import ZipFile
from functools import reduce

import seaborn as sns

from sklearn.utils.validation import column_or_1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from pandas.plotting import scatter_matrix

In [ ]:
#default paths to files generated with data_splitting
DATA_BASE = '2022.11.07-merged-single-observation/'
PATHS = ['100', '1000', '1500','2000', '2500', 'average', 'deep', 'standard']
DATA_FILE = '/CLEANED_MERGED_DATA.csv'
DATA_PATH = DATA_BASE + PATHS[6] + DATA_FILE

In [ ]:
#DATAPATH is path to your file
df = pd.read_csv(DATA_PATH,index_col=False)

In [ ]:
#drop date and normalize rest of columns
df = df.drop(['DATE'], axis=1)
cols = list(set(df.columns))
df[cols] -= df[cols].min()
df[cols] /= df[cols].max()

In [ ]:
# quality is linear combitnation of few other parameters
# we used elements occuring in fertilizers and SIO2 (silicate) which comes from rivers as X

X_cols = ['TP', 'TN', 'NO3','NO2','PO4', 'NH4', 'SIO2']
df['quality'] = 0.4*df['OXY'] + 0.3*df['SECCHI'] + 0.3*df['CHLORA']
y_col = ['quality']

X = df[X_cols]
y = df[y_col]
y = column_or_1d(y, warn=True)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true  ) ) ) * 100

In [ ]:
# linear regression
lg = LinearRegression()
lg.fit(x_train, y_train)
y_pred = lg.predict(x_test)
print(mean_squared_error(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))
print(mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
# Gradient boosting regression
bg = GradientBoostingRegressor()
bg.fit(x_train, y_train)
y_pred = bg.predict(x_test)
print(mean_squared_error(y_test, y_pred))
print(mean_absolute_error(y_test,y_pred))
print(mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
# neural netword
mlp = MLPRegressor(activation='logistic', hidden_layer_sizes=(2,2), solver='sgd', max_iter=3000)
mlp.fit(x_train,y_train)
y_pred = mlp.predict(x_test)
print(mean_squared_error(y_test, y_pred))
print(mean_absolute_error(y_test,y_pred))
print(mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
# random forest 
rfr = RandomForestRegressor(random_state=0)
rfr.fit(x_train, y_train)
y_pred = rfr.predict(x_test)
print(mean_squared_error(y_test, y_pred))
print(mean_absolute_error(y_test,y_pred))
print(mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
feat_lebels = x_train.columns[0:]
importances  = rfr.feature_importances_
indices = np.argsort(importances)[::-1]
index_list = []
value_list = []
for i,j in zip(range(x_train.shape[1]-1),indices):
    index_list.append(feat_lebels[j])
    value_list.append(importances[j])
    print(i+1, feat_lebels[j], importances[j])

plt.figure(figsize=(10,6))
plt.barh(index_list[::-1],value_list[::-1])
plt.yticks(fontsize = 12)
plt.title('feature',fontsize = 14)
plt.show()

In [ ]:
# find best parameters
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators':[50,75,100],
    'min_samples_split':[2,3,5],
    'min_samples_leaf':[1,2,3]
}
grid_seach_rf= GridSearchCV(estimator=RandomForestRegressor(random_state=0),param_grid=param_grid, scoring='neg_mean_squared_error', cv = 5)
grid_seach_rf.fit(x_train, y_train)
best = grid_seach_rf.best_params_
print(best)

In [ ]:
best

In [ ]:
# model
rfr_ = RandomForestRegressor(n_estimators=best['n_estimators'], min_samples_leaf=best['min_samples_leaf'], min_samples_split=best['min_samples_split'], random_state=0)
rfr_.fit(x_train, y_train)
y_pred = rfr_.predict(x_test)
print(mean_squared_error(y_test, y_pred))
print(mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
# score

from sklearn.metrics import r2_score
regressor = RandomForestRegressor(n_estimators=100, min_samples_leaf=5)
regressor.fit(x_train, y_train) 
y_pred = regressor.predict(x_test)
print('sklearn score:{}'.format(r2_score(y_test, y_pred)))  

errors = abs(y_pred - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:\n', np.round(np.mean(errors), 2), 'degrees.')
mape = 100 * (errors / y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:\n', round(accuracy, 2), '%.')

In [ ]:
col = y_pred.shape[0]
y_pred = pd.DataFrame(y_pred[-col:], index=range(col), columns=['quality'])

In [ ]:
y_test.shape[0]

In [ ]:
y_pred

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(y_test[150:200], color='red', label='quality_test')
plt.plot(y_pred[150:200], color='blue', label='quality_pred')
plt.legend(loc="upper right")
plt.title("quality")
plt.show()
